<h1 style="margin: 0; padding: 0; color: #d9f4e4;">Import Libraries<h1/>

In [1]:
import glob
import pandas as pd
import re
import html
from sentence_transformers import SentenceTransformer
import numpy as np

2025-11-16 12:09:30.709357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763294970.890109      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763294970.937660      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

<h1 style="margin: 0; padding: 0; color: #d9f4e4;">Global Variables<h1/>

In [2]:
PATH = "/kaggle/input/hyb-reco-amazone/Dataset"
FASHION_FILE = "/AMAZON_FASHION.json"
META_FILE = "/meta_AMAZON_FASHION.json"

In [3]:
TEXT_COLS = ["title", "brand", "description", "feature", "summary", "reviewText", "details", "fit", "rank", "tech1"]
LIST_COLS = ["title", "des", "similar_item"]
IMAGE_COLS = ["imageURL", "imageURLHighRes"]
NEEDED_COLS = ["asin", "title", "brand", "description", "feature", "details", "fit", "reviewText", "summary", "imageURL", "price", "overall"]

In [4]:
transformer_model_name = "BAAI/bge-large-en-v1.5"

<h1 style="margin: 0; padding: 0; color: #d9f4e4;">Functions<h1/>

In [5]:
def normalize_list_field(x):
    if isinstance(x, list):
        return " ".join(str(i) for i in x if i is not None)
    if pd.isna(x):
        return ""
    return str(x)

In [6]:
def clean(text):
    if text is None:
        return ""
    text = str(text)
    text = html.unescape(text)

    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'http\S+|www.\S+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

<h1 style="margin: 0; padding: 0; color: #d9f4e4;">Import data & Display Data<h1/>

In [7]:
fashion_path = PATH + FASHION_FILE
meta_path = PATH + META_FILE

In [8]:
fashion_df = pd.read_json(fashion_path, lines=True)
meta_df = pd.read_json(meta_path, lines=True)

In [9]:
fashion_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883636 entries, 0 to 883635
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         883636 non-null  int64  
 1   verified        883636 non-null  bool   
 2   reviewTime      883636 non-null  object 
 3   reviewerID      883636 non-null  object 
 4   asin            883636 non-null  object 
 5   reviewerName    883544 non-null  object 
 6   reviewText      882403 non-null  object 
 7   summary         883103 non-null  object 
 8   unixReviewTime  883636 non-null  int64  
 9   vote            79900 non-null   float64
 10  style           304569 non-null  object 
 11  image           28807 non-null   object 
dtypes: bool(1), float64(1), int64(2), object(8)
memory usage: 75.0+ MB


In [10]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186637 entries, 0 to 186636
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   title            186632 non-null  object
 1   brand            139957 non-null  object
 2   feature          123875 non-null  object
 3   rank             180222 non-null  object
 4   date             185001 non-null  object
 5   asin             186637 non-null  object
 6   imageURL         132017 non-null  object
 7   imageURLHighRes  132017 non-null  object
 8   description      15869 non-null   object
 9   price            17799 non-null   object
 10  also_view        11595 non-null   object
 11  also_buy         21642 non-null   object
 12  fit              4831 non-null    object
 13  details          885 non-null     object
 14  similar_item     317 non-null     object
 15  tech1            97 non-null      object
dtypes: object(16)
memory usage: 22.8+ MB


In [11]:
fashion_df.head(3)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN


In [12]:
meta_df.head(3)

,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
fashion_df.isnull().sum()

overall                0
verified               0
reviewTime             0
reviewerID             0
asin                   0
reviewerName          92
reviewText          1233
summary              533
unixReviewTime         0
vote              803736
style             579067
image             854829
dtype: int64

In [14]:
meta_df.isnull().sum()

title                   5
brand               46680
feature             62762
rank                 6415
date                 1636
asin                    0
imageURL            54620
imageURLHighRes     54620
description        170768
price              168838
also_view          175042
also_buy           164995
fit                181806
details            185752
similar_item       186320
tech1              186540
dtype: int64

In [15]:
df = fashion_df.merge(meta_df, on="asin", how="left")
df = df[NEEDED_COLS]

<h1 style="margin: 0; padding: 0; color: #d9f4e4;">Pre-processing<h1/>

In [16]:
for col in TEXT_COLS:
    if col in df.columns:
        df[col] = df[col].fillna("").apply(clean)

for col in LIST_COLS:
    if col in df.columns:
        df[col] = df[col].apply(normalize_list_field).apply(clean)

avg_rating = df["overall"].mean()
df["overall"] = df["overall"].fillna(avg_rating)
df["overall"] = df["overall"].astype(float)

In [17]:
def build_document(row):
    parts = []

    if "title" in row and row["title"]:
        parts.append(f"Title: {row['title']}")
    if "brand" in row and row["brand"]:
        parts.append(f"Brand: {row['brand']}")
    if "description" in row and row["description"]:
        parts.append(f"Description: {row['description']}")
    if "feature" in row and row["feature"]:
        parts.append(f"Features: {row['feature']}")
    if "details" in row and row["details"]:
        parts.append(f"Details: {row['details']}")
    if "fit" in row and row["fit"]:
        parts.append(f"Fit: {row['fit']}")
    if "summary" in row and row["summary"]:
        parts.append(f"Review Summary: {row['summary']}")
    if "reviewText" in row and row["reviewText"]:
        parts.append(f"Review: {row['reviewText']}")
    return " | ".join(parts)


df["document_text"] = df.apply(build_document, axis=1)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884806 entries, 0 to 884805
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   asin           884806 non-null  object 
 1   title          884806 non-null  object 
 2   brand          884806 non-null  object 
 3   description    884806 non-null  object 
 4   feature        884806 non-null  object 
 5   details        884806 non-null  object 
 6   fit            884806 non-null  object 
 7   reviewText     884806 non-null  object 
 8   summary        884806 non-null  object 
 9   imageURL       717533 non-null  object 
 10  price          297772 non-null  object 
 11  overall        884806 non-null  float64
 12  document_text  884806 non-null  object 
dtypes: float64(1), object(12)
memory usage: 87.8+ MB


In [19]:
df.head(3)

,asin,title,brand,description,feature,details,fit,reviewText,summary,imageURL,price,overall,document_text
0,7106116521,Milliongadgets(TM) Earring Safety Backs For Fi...,,,,,,Exactly what I needed.,perfect replacements!!,[https://images-na.ssl-images-amazon.com/image...,NaN,5.0,Title: Milliongadgets(TM) Earring Safety Backs...
1,7106116521,Milliongadgets(TM) Earring Safety Backs For Fi...,,,,,,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",[https://images-na.ssl-images-amazon.com/image...,NaN,2.0,Title: Milliongadgets(TM) Earring Safety Backs...
2,7106116521,Milliongadgets(TM) Earring Safety Backs For Fi...,,,,,,Love these... I am going to order another pack...,My New 'Friends' !!,[https://images-na.ssl-images-amazon.com/image...,NaN,4.0,Title: Milliongadgets(TM) Earring Safety Backs...


<h1 style="margin: 0; padding: 0; color: #d9f4e4;">Empedding<h1/>

In [20]:
model = SentenceTransformer(transformer_model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [21]:
texts = df["document_text"].tolist()

<h2 style="margin: 0; padding: 0; color: #00f;">Encoding<h2/>

In [ ]:
embeddings = model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

In [ ]:
embeddings.shape

In [ ]:
embeddings

<h1 style="margin: 0; padding: 0; color: #d9f4e4;">Transformers Model<h1/>